Esse código usa o seguinte dataset do kaggle: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

Para testar, baixe o arquivo csv e salve na mesma pasta que o código com o nome "creditcard.csv"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('creditcard.csv')
df

In [ ]:
X = df.drop(columns=['Class']) #features
y = df['Class'] #target

#padronizar os dados
scaler = StandardScaler()
X_sca = scaler.fit_transform(X)

#redução de dimensionalidade (PCA) - simplifica a visualização e eficiência
pca = PCA(n_components=2) #novo conjunto de dados com apenas duas dimensões, pegam a maior parte da variação
X_pca = pca.fit_transform(X_sca)
print('componentes principais:')
print(pca.components_)

#K-means 
kmeans = KMeans(n_clusters=2, random_state=42) #dois clusters, um para fraude e outro não fraude
predict = kmeans.fit_predict(X_pca) #encontrando os dois centros
df['Cluster'] = predict

#matriz de confusão para determinar qual cluster é de fraude
matrix = pd.crosstab(df['Class'], df['Cluster'], rownames=['Class'], colnames=['Cluster']) #cria a tabela (class x cluster)
print(matrix)
fraude_cluster = matrix.idxmax(axis=1)[1] #qual cluster tem mais fraudes

print(f'O cluster que representa fraudes é: {fraude_cluster}')

# Printa a inércia do modelo
print(f'Inercia do modelo: {kmeans.inertia_}')

In [ ]:
# Cria df auxiliar para plotagem do gráfico
df_teste = pd.DataFrame(X_pca)
df_teste['class'] = df['Class']
df_teste['cluster'] = predict
df_teste


df0 = df_teste[df_teste['cluster'] == 0] # dados apenas classificados como 0
df1 = df_teste[df_teste['cluster'] == 1] # dados apenas classificados como 1

plt.scatter(df1.iloc[:,0], df1.iloc[:,1], color='red') # plota o agrupamento de dados 1
plt.scatter(df0.iloc[:,0], df0.iloc[:,1], color='blue') # plota o agrupamento de dados 0

plt.ylabel('Component 1')
plt.xlabel('Component 2')
#plt.legend()


In [ ]:
# Cria gráfico de cotovelo
from sklearn.cluster import KMeans
k_range = range(1,10)
inertias = []
for k in k_range:
    kn = KMeans(n_clusters=k, random_state=0)
    kn.fit(X_pca)
    inertias.append(kn.inertia_)

plt.plot(k_range, inertias)
plt.ylabel('Inercia do modelo')
plt.xlabel('Cluster')
plt.title('Elbow Method')